# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 32

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [8]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 10'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
411,Patrício,Wolves,53,31.904762,134,142.048430,4.2,4.452264,4.408282,4.669268,1.933940
340,Lloris,Tottenham,53,14.883721,64,65.791637,4.3,4.420376,4.002040,4.551735,1.920088
471,Henderson,Sheffield Utd,52,29.782609,137,124.818133,4.6,4.190974,4.186582,4.568883,1.837864
93,Pope,Burnley,50,31.956522,147,132.742507,4.6,4.153847,4.973442,4.956112,1.857657
168,Schmeichel,Leicester City,54,31.860465,137,131.745594,4.3,4.135081,5.391438,5.477872,1.779455
131,Guaita,Crystal Palace,49,29.069767,125,116.590868,4.3,4.010726,5.346537,4.882290,1.811861
235,de Gea,Manchester Utd,53,31.621622,117,125.764885,3.7,3.977180,4.554989,4.262818,1.727577
189,Alisson,Liverpool,62,22.826087,105,87.360665,4.6,3.827229,2.667045,3.287919,1.537052
603,Reina,Aston Villa,43,5.937500,19,22.630816,3.2,3.811506,1.464012,3.677821,1.838071


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
182,Alexander-Arnold,Liverpool,78,32.241379,187,171.902738,5.8,5.331743,3.704769,4.572821,1.909070
401,Doherty,Wolves,64,30.000000,141,144.272137,4.7,4.809071,6.053464,5.960373,1.900952
103,Alonso,Chelsea,62,13.965517,81,66.184312,5.8,4.739124,3.909462,4.760648,1.903278
181,Robertson,Liverpool,70,29.795918,146,139.601696,4.9,4.685262,1.606625,3.257061,1.770863
105,Azpilicueta,Chelsea,59,30.000000,102,132.151412,3.4,4.405047,4.314843,4.353723,1.813529
297,Lundstram,Sheffield Utd,48,29.777778,134,128.828186,4.5,4.326320,3.089761,3.300638,1.974686
183,van Dijk,Liverpool,65,31.836735,156,132.682967,4.9,4.167606,2.905710,3.526047,1.634670
122,Wan-Bissaka,Manchester Utd,55,29.189189,108,118.634457,3.7,4.064329,6.033914,5.414050,1.733036
405,Boly,Wolves,48,16.122449,79,65.106838,4.9,4.038272,4.781120,4.320597,1.843211


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,126,28.028169,199,188.452309,7.1,6.723675,5.551837,5.699198,1.894181
215,De Bruyne,Manchester City,107,29.861111,215,192.613059,7.2,6.450298,9.325324,7.900675,1.971914
618,Fernandes,Manchester Utd,88,8.024691,65,48.349332,8.1,6.025071,5.713466,6.365751,2.031051
192,Mané,Liverpool,125,29.076923,189,169.963976,6.5,5.845322,4.738318,4.617537,1.653307
214,Sterling,Manchester City,117,27.037037,146,156.234938,5.4,5.778553,7.204948,6.705948,1.689375
342,Son,Tottenham,98,23.859649,136,120.904213,5.7,5.067309,4.865516,5.424065,1.618692
239,Martial,Manchester Utd,82,25.964912,148,126.445903,5.7,4.869876,6.298044,5.474760,1.700634
431,Pulisic,Chelsea,71,19.166667,92,90.745944,4.8,4.734571,6.248320,6.289764,1.776853
217,Mahrez,Manchester City,85,27.090909,149,127.696884,5.5,4.713643,4.234153,4.602874,1.616766


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
233,Rashford,Manchester Utd,91,24.915254,147,140.976255,5.9,5.658231,5.055759,5.102551,1.875685
166,Vardy,Leicester City,97,28.833333,173,150.231076,6.0,5.210326,3.144464,3.274675,1.672935
409,Jiménez,Wolves,83,31.923077,166,160.791076,5.2,5.036829,4.341508,4.603633,1.748309
11,Aubameyang,Arsenal,110,30.000000,174,149.640717,5.8,4.988024,6.211832,5.695549,1.503946
210,Agüero,Manchester City,117,24.000000,132,117.091076,5.5,4.878795,4.567077,4.398577,1.426329
460,Abraham,Chelsea,75,28.163265,138,133.731076,4.9,4.748422,4.226759,3.971866,1.733879
187,Firmino,Liverpool,95,32.380952,136,152.841434,4.2,4.720103,3.293122,3.944184,1.531404
313,Ings,Southampton,73,31.923077,166,145.210359,5.2,4.548758,4.736767,4.027763,1.683571
278,Pukki,Norwich City,65,31.190476,131,137.855538,4.2,4.419796,1.930754,2.771453,1.733587


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,126,28.028169,199,188.452309,7.1,6.723675,5.551837,5.699198,1.894181
215,De Bruyne,Manchester City,107,29.861111,215,192.613059,7.2,6.450298,9.325324,7.900675,1.971914
618,Fernandes,Manchester Utd,88,8.024691,65,48.349332,8.1,6.025071,5.713466,6.365751,2.031051
192,Mané,Liverpool,125,29.076923,189,169.963976,6.5,5.845322,4.738318,4.617537,1.653307
214,Sterling,Manchester City,117,27.037037,146,156.234938,5.4,5.778553,7.204948,6.705948,1.689375
233,Rashford,Manchester Utd,91,24.915254,147,140.976255,5.9,5.658231,5.055759,5.102551,1.875685
182,Alexander-Arnold,Liverpool,78,32.241379,187,171.902738,5.8,5.331743,3.704769,4.572821,1.909070
166,Vardy,Leicester City,97,28.833333,173,150.231076,6.0,5.210326,3.144464,3.274675,1.672935
342,Son,Tottenham,98,23.859649,136,120.904213,5.7,5.067309,4.865516,5.424065,1.618692


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
215,De Bruyne,Manchester City,107,29.861111,215,192.613059,7.2,6.450298,9.325324,7.900675,1.971914
214,Sterling,Manchester City,117,27.037037,146,156.234938,5.4,5.778553,7.204948,6.705948,1.689375
618,Fernandes,Manchester Utd,88,8.024691,65,48.349332,8.1,6.025071,5.713466,6.365751,2.031051
257,Lejeune,Newcastle Utd,43,6.000000,24,17.788601,4.0,2.964767,NaN,6.335912,1.429738
431,Pulisic,Chelsea,71,19.166667,92,90.745944,4.8,4.734571,6.248320,6.289764,1.776853
401,Doherty,Wolves,64,30.000000,141,144.272137,4.7,4.809071,6.053464,5.960373,1.900952
389,Antonio,West Ham,69,18.181818,60,76.635558,3.3,4.214956,7.301038,5.939437,1.604606
402,Jonny,Wolves,54,29.411765,100,108.396970,3.4,3.685497,6.953425,5.774583,1.585985
191,Salah,Liverpool,126,28.028169,199,188.452309,7.1,6.723675,5.551837,5.699198,1.894181


In [11]:
metric = 'valuePoints metric'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,88,8.024691,65,48.349332,8.1,6.025071,5.713466,6.365751,2.031051
297,Lundstram,Sheffield Utd,48,29.777778,134,128.828186,4.5,4.326320,3.089761,3.300638,1.974686
215,De Bruyne,Manchester City,107,29.861111,215,192.613059,7.2,6.450298,9.325324,7.900675,1.971914
411,Patrício,Wolves,53,31.904762,134,142.048430,4.2,4.452264,4.408282,4.669268,1.933940
340,Lloris,Tottenham,53,14.883721,64,65.791637,4.3,4.420376,4.002040,4.551735,1.920088
182,Alexander-Arnold,Liverpool,78,32.241379,187,171.902738,5.8,5.331743,3.704769,4.572821,1.909070
103,Alonso,Chelsea,62,13.965517,81,66.184312,5.8,4.739124,3.909462,4.760648,1.903278
401,Doherty,Wolves,64,30.000000,141,144.272137,4.7,4.809071,6.053464,5.960373,1.900952
191,Salah,Liverpool,126,28.028169,199,188.452309,7.1,6.723675,5.551837,5.699198,1.894181
